# Solving the Inverse Spring Mass Problem

Another important advantage of a neural network solver over traditional numerical methods is its ability to solve inverse problems. In an inverse problem, we start with a set of observations and then use those observations to calculate the causal factors that produced them. To demonstrate this concept we will solve the same spring mass system in an inverse setting. Specifically, we will assume that the data for the displacements of the 3 masses $x_1, x_2 \text{ and } x_3$ is available to us through an analytical solution and the first mass ($m_1$) and the last spring coefficient ($k_4$) is unknown. We will use the neural network to assimilate the displacements from the analytical solution and use it to invert out the unknown quantities from ODEs.

## Case Setup

The equation file again remains the same for this part. Here, we will use the [`PointwiseConstraint`](https://docs.nvidia.com/deeplearning/modulus/api/modulus.domain.constraint.html#modulus.domain.constraint.continuous.PointwiseConstraint) class to assimilate the solution in the training data. We will make a network to memorize the three displacements by developing a mapping between $t$ and $(x_1, x_2, x_3)$. The second network will be trained to invert out the desired quantities viz. $(m_1, k_4)$.

### Note

Now we will describe the contents of the [`spring_mass_inverse.py`](../../source_code/spring_mass/spring_mass_inverse.py) script.

## Defining the Equations, Networks and Nodes for an Inverse Problem

Since part of the problem involves memorizing the given solution, we will have `'t'` as an input NumPy array and `'x_1', 'x_2', 'x_3', 'ode_x1', 'ode_x2', 'ode_x3'` as the output NumPy arrays. Setting `'x_1', 'x_2', 'x_3'` as input values from analytical data, we are essentially making the network assimilate the analytical distribution of these variables in the selected domain. Setting `'ode_x1', 'ode_x2', 'ode_x3'` equal to 0, we will also inform the network to satisfy the ODE losses at those sampled points. Now, assuming that we also know the initial conditions of the solution, except the `'m_1'` and `'k_4'`, all the variables in these ODEs are known. Thus, the network can use this information to invert out the unknowns. The below code shows the data preparation step.

```python
import torch
import numpy as np
from sympy import Symbol, Eq

import modulus
from modulus.hydra import ModulusConfig, instantiate_arch
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_1d import Point1D
from modulus.geometry import Parameterization
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseConstraint,
)
from modulus.domain.validator import PointwiseValidator
from modulus.domain.monitor import PointwiseMonitor
from modulus.key import Key
from modulus.node import Node

from spring_mass_ode import SpringMass

@modulus.main(config_path="conf", config_name="config_inverse")
def run(cfg: ModulusConfig) -> None:
    # prepare data
    t_max = 10.0
    deltaT = 0.01
    t = np.arange(0, t_max, deltaT)
    t = np.expand_dims(t, axis=-1)

    invar_numpy = {"t": t}
    outvar_numpy = {
        "x1": (1 / 6) * np.cos(t)
        + (1 / 2) * np.cos(np.sqrt(3) * t)
        + (1 / 3) * np.cos(2 * t),
        "x2": (2 / 6) * np.cos(t)
        + (0 / 2) * np.cos(np.sqrt(3) * t)
        - (1 / 3) * np.cos(2 * t),
        "x3": (1 / 6) * np.cos(t)
        - (1 / 2) * np.cos(np.sqrt(3) * t)
        + (1 / 3) * np.cos(2 * t),
    }
    outvar_numpy.update({"ode_x1": np.full_like(invar_numpy["t"], 0)})
    outvar_numpy.update({"ode_x2": np.full_like(invar_numpy["t"], 0)})
    outvar_numpy.update({"ode_x3": np.full_like(invar_numpy["t"], 0)})
```

The process of creating neural networks for an inverse problem is similar to other problems we have seen in previous sections. However, as the information for the displacements, and in turn their gradients (i.e. velocities and accelerations), is already present (from analytical data) for the network to memorize, we will detach these variables and computation graph in their respective equations. This means that only the networks predicting `'m1'` and `'k4'` will be optimized to minimize the equation residuals. The displacements, velocities, and accelerations are treated as ground truth data.

Also, note that the mass and spring constant are passed in as Symbolic variables (`'m1'` and `'k4'` respectively) to the
equations as they are unknowns in this problem.

```python
    # make list of nodes to unroll graph on
    sm = SpringMass(k=(2, 1, 1, "k4"), m=("m1", 1, 1))
    sm_net = instantiate_arch(
        input_keys=[Key("t")],
        output_keys=[Key("x1"), Key("x2"), Key("x3")],
        cfg=cfg.arch.fully_connected,
    )
    invert_net = instantiate_arch(
        input_keys=[Key("t")],
        output_keys=[Key("m1"), Key("k4")],
        cfg=cfg.arch.fully_connected,
    )

    nodes = (
        sm.make_nodes(
            detach_names=[
                "x1",
                "x1__t",
                "x1__t__t",
                "x2",
                "x2__t",
                "x2__t__t",
                "x3",
                "x3__t",
                "x3__t__t",
            ]
        )
        + [sm_net.make_node(name="spring_mass_network", jit=cfg.jit)]
        + [invert_net.make_node(name="invert_network", jit=cfg.jit)]
    )
```

For this problem, we will use the solution coming from the analytical solution in the form of the NumPy arrays we defined earlier. We will use the `PointwiseConstraint` class to handle such input data. The `PointwiseConstraint` class takes in separate dictionaries for input variables and output variables. These dictionaries have a key for each variable and a NumPy array of values associated to the key. The below code shows the addition of the boundary constraint and the constraint coming from the NumPy arrays we just defined.

```python
    # add constraints to solver
    # make geometry
    geo = Point1D(0)
    t_symbol = Symbol("t")
    x = Symbol("x")
    time_range = {t_symbol: (0, t_max)}

    # make domain
    domain = Domain()

    # initial conditions
    IC = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"x1": 1.0, "x2": 0, "x3": 0, "x1__t": 0, "x2__t": 0, "x3__t": 0},
        batch_size=cfg.batch_size.IC,
        lambda_weighting={
            "x1": 1.0,
            "x2": 1.0,
            "x3": 1.0,
            "x1__t": 1.0,
            "x2__t": 1.0,
            "x3__t": 1.0,
        },
        parameterization=Parameterization({t_symbol: 0}),
    )
    domain.add_constraint(IC, name="IC")

    # data and pdes
    data = PointwiseConstraint.from_numpy(
        nodes=nodes,
        invar=invar_numpy,
        outvar=outvar_numpy,
        batch_size=cfg.batch_size.data,
    )
    domain.add_constraint(data, name="Data")
```

## Adding Monitors

For this section, we will monitor the convergence of average `'m_1'` and `'k_4'` inside the domain as the solution progresses. Once we find that the average value of these quantities have reached a steady value, we can end the simulation. The code to generate such monitors can be found below.

```python
    # add monitors
    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["m1"],
        metrics={"mean_m1": lambda var: torch.mean(var["m1"])},
        nodes=nodes,
    )
    domain.add_monitor(monitor)

    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["k4"],
        metrics={"mean_k4": lambda var: torch.mean(var["k4"])},
        nodes=nodes,
    )
    domain.add_monitor(monitor)
```

## Solver and Training

Now that we have the definitions for the various constraints and domains complete, we can form the solver and run the problem similar to before. The code to do the same can be found below.

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

In [ ]:
%load_ext tensorboard
%tensorboard --logdir outputs/spring_mass_inverse

In [ ]:
!python3 ../../source_code/spring_mass/spring_mass_inverse.py

## Visualizing the Solution

We can monitor the Tensorboard plots to see the convergence of the simulation and the final values of the $m_1$ and $k_4$. We see that after sufficient training, we get the $m_1$ and $k_4$ as 1 and 2 respectively. These were the same values that we used to create the analytical solution.

## Next

In the next notebook we will cover some theory behind the Fourier Neural Operator (FNO) and some of its variants. We will then apply these architectures on a Darcy flow field prediction problem in a data-driven fashion.

Please continue to [the next notebook](../Operators/FNO_Darcy.ipynb).